<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-2/Session_2_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD


In [0]:
# https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
# https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip





In [0]:
import os

!wget --no-check-certificate \
    https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip \
    -O /tmp/smallData.zip

In [0]:
import zipfile
def unzip(file,target):
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall(target)
  zip_ref.close()

unzip("/tmp/smallData.zip","/tmp")

In [5]:
if os.path.exists('/tmp/smallData'):
    print(os.listdir('/tmp/smallData'))


['test', 'train', '.gitignore']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)
CLASSES_COUNT = 2;
epochs = 50

In [8]:
#define the sub folders for both training and test
training = os.path.join("/tmp/smallData",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=20,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           )

Found 95 images belonging to 2 classes.


In [10]:
model = Sequential()
model.add(Flatten(input_shape=targetSize_withdepth))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(CLASSES_COUNT,activation='softmax'))
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              30721024  
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 1026      
Total params: 31,246,850
Trainable params: 31,246,850
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_generator,epochs=epochs,steps_per_epoch=20)


Epoch 1/50


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


20/20 [==============================] - 2s 103ms/step - loss: 1.4548 - accuracy: 0.7421
Epoch 2/50
20/20 [==============================] - 2s 106ms/step - loss: 0.0864 - accuracy: 0.9789
Epoch 3/50
20/20 [==============================] - 2s 110ms/step - loss: 0.0290 - accuracy: 1.0000
Epoch 4/50
20/20 [==============================] - 2s 108ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 5/50
20/20 [==============================] - 2s 103ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 6/50
20/20 [==============================] - 2s 108ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 7/50
20/20 [==============================] - 2s 108ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 8/50
20/20 [==============================] - 2s 109ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 9/50
20/20 [==============================] - 2s 110ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 10/50
20/20 [==============================] - 2s 111ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 11/5

In [0]:
from sklearn.metrics import confusion_matrix, classification_report
def test(generator, model):
  predictions = model.predict(generator, steps=1)
  row_index = predictions.argmax(axis=1)
  target_names = generator.class_indices.keys()
  print(target_names)
  print(confusion_matrix(generator.classes, row_index))
  print('Classification Report')
  print(classification_report(generator.classes, row_index, target_names=target_names))

In [14]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/smallData/train",
                                                         target_size=(100,100),
                                                         batch_size=95,
                                                         shuffle=False)
test(generator=test_generator, model=model)

Found 95 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


dict_keys(['bar_chart', 'pie_chart'])
[[39  0]
 [ 0 56]]
Classification Report
              precision    recall  f1-score   support

   bar_chart       1.00      1.00      1.00        39
   pie_chart       1.00      1.00      1.00        56

    accuracy                           1.00        95
   macro avg       1.00      1.00      1.00        95
weighted avg       1.00      1.00      1.00        95



In [15]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/smallData/test",
                                                         target_size=(100,100),
                                                         batch_size=37,
                                                         shuffle=False)

Found 37 images belonging to 2 classes.


In [16]:
test(generator=test_generator, model=model)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


dict_keys(['bar_chart', 'pie_chart'])
[[15  4]
 [ 4 14]]
Classification Report
              precision    recall  f1-score   support

   bar_chart       0.79      0.79      0.79        19
   pie_chart       0.78      0.78      0.78        18

    accuracy                           0.78        37
   macro avg       0.78      0.78      0.78        37
weighted avg       0.78      0.78      0.78        37

